In [1]:
import pandas as pd

In [2]:
film = pd.read_csv('movies.csv')

In [3]:
film.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [4]:
film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7668 non-null   object 
 1   rating    7591 non-null   object 
 2   genre     7668 non-null   object 
 3   year      7668 non-null   int64  
 4   released  7666 non-null   object 
 5   score     7665 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7668 non-null   object 
 8   writer    7665 non-null   object 
 9   star      7667 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7479 non-null   float64
 13  company   7651 non-null   object 
 14  runtime   7664 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 898.7+ KB


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [6]:
sentences = film['name'].astype(str).tolist()

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

In [8]:
input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [9]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [12]:
model = Sequential()
model.add(Embedding(total_words, 64))
model.add(SimpleRNN(64))
model.add(Dense(total_words, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(xs, ys, epochs=100, verbose=1)


Epoch 1/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0574 - loss: 7.9222
Epoch 2/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.0875 - loss: 6.9242
Epoch 3/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1018 - loss: 6.5424
Epoch 4/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.1068 - loss: 6.2839
Epoch 5/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.1117 - loss: 6.0483
Epoch 6/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1280 - loss: 5.7651
Epoch 7/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1436 - loss: 5.5084
Epoch 8/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1570 - loss: 5.2927
Epoch 9/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.1767 - loss: 5.0709
Epoch 10/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.1972 - loss: 4.8699
Epoch 11/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.2247 - loss: 4.6527
Epoch 12/100
405/405 ━━━━━━━━━

405/405 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6593 - loss: 1.4149
Epoch 68/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6548 - loss: 1.3857
Epoch 69/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6532 - loss: 1.4060
Epoch 70/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6539 - loss: 1.3960
Epoch 71/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6593 - loss: 1.3956
Epoch 72/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6560 - loss: 1.3998
Epoch 73/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6505 - loss: 1.3712
Epoch 74/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6646 - loss: 1.3844
Epoch 75/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6590 - loss: 1.3668
Epoch 76/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6570 - loss: 1.3763
Epoch 77/100
405/405 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6526 - loss: 1.3778
Epoch 78/100
405/405 ━━━━━━━━━━━━━

In [16]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)[0]
    return tokenizer.index_word[predicted_word_index]

In [17]:
seed_text = "Star"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Prediksi kata berikutnya: {next_word}")

Prediksi kata berikutnya: wars
